## group memebers: Nafiseh Mohammadi, Ali Saghi

In [1]:
# Importing very basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import json

# 1) Reading the Tweets in Python

In [2]:
# Reading the JSON lines in truncated-untruncated format
data = []
with open("intro-to-nlp/english-tweets-sample.jsonl","rt",encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))
    
print("Data length:",len(data))
print("Data type:",type(data))
print("First item type:",type(data[0]))
print("First item:",data[0])

Data length: 10000
Data type: <class 'list'>
First item type: <class 'dict'>
First item: {'created_at': 'Tue Dec 26 14:16:22 +0000 2017', 'id': 945659557480611840, 'id_str': '945659557480611840', 'text': 'Check out my class in #GranblueFantasy! https://t.co/pAvXn8diJr', 'display_text_range': [0, 39], 'source': '<a href="http://granbluefantasy.jp/" rel="nofollow">グランブルー ファンタジー</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 883980236655779840, 'id_str': '883980236655779840', 'name': 'Pc Kwok', 'screen_name': 'jensenpck', 'location': None, 'url': None, 'description': None, 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 0, 'friends_count': 1, 'listed_count': 0, 'favourites_count': 0, 'statuses_count': 42, 'created_at': 'Sun Jul 09 09:24:46 +0000 2017', 'utc_offset': None, 'time_zone': None, 'geo_enab

In [3]:
# Different fields inside each JSON file
print(data[0].keys())

dict_keys(['created_at', 'id', 'id_str', 'text', 'display_text_range', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'extended_entities', 'favorited', 'retweeted', 'possibly_sensitive', 'filter_level', 'lang', 'timestamp_ms'])


# 2) Extract the actual Text fields from the tweet JSONs

In [4]:
# Reading the full-text of JSON lines when it is in truncated format
truncated_data = []
for item in data:
    if item['truncated'] == True:
        truncated_data.append(item['extended_tweet']['full_text'].lower().split('https:')[0])
    else:
        truncated_data.append(item['text'].lower().split('https:')[0])
    
print("Truncated_data length:",len(truncated_data))
print("Truncated_data type:",type(truncated_data))
print("First item type:",type(truncated_data[5463]))
print("First item:",truncated_data[5463])

Truncated_data length: 10000
Truncated_data type: <class 'list'>
First item type: <class 'str'>
First item: rt @oeasywayout: s/o to @mrmakeitcold for the trade💪💯 


In [5]:
# Create a data frame from the list of texts
df_text = pd.DataFrame(truncated_data,columns=["Text"])
df_text

,Text
0,check out my class in #granbluefantasy!
1,extending a big thank you to our community par...
2,blueberry 🍨
3,rt @liluzivert: bad day ☹️®️
4,@prologve_ @bts_army @bts_twt i'm chim tho
...,...
9995,rt @chicodustyy: y'all try to cancel cardi eve...
9996,"#russiagatecloses will they flee? or, stand t..."
9997,i am really interested in the small ring of pi...
9998,rt @mahraalhosani: no one will ever care like ...


In [6]:
# Text segmentation with Naive method 1: Split from whitespace characters
text = df_text.iloc[0].to_string()
tokenized_text = text.split() # split(): Return a list of the words in the string, using whitespace as the delimiter string.
for w in tokenized_text:
    print(w)

Text
check
out
my
class
in
#granbluefantasy!


In [7]:
# Text segmentation with Naive method 2: Split from whitespace characters, take into account punctuation
import re

text = df_text.iloc[0].to_string()
tokenized = re.sub(r'([.,!?]+)',r' \1',text) # replace . , ! ? with whitespace+character(s), '+' means one or more
tokenized = re.sub(r"(n't)",r" \1",tokenized) # clitics
print(tokenized) # Note: this is still string, apply simple whitespace splitting to get a list of tokens

Text    check out my class in #granbluefantasy ! 


# 3) Segment each Tweet

In [8]:
#!pip3 install ufal.udpipe
import ufal.udpipe as udpipe

model = udpipe.Model.load("intro-to-nlp/en.segmenter.udpipe")
# horizontal: returns one sentence per line, with words separated by a single space
pipeline = udpipe.Pipeline(model,"tokenize","none","none","horizontal") 

segmented_document = pipeline.process(truncated_data[-1])

print(segmented_document)

rt @ diosaslesbianas : girl beautiful tits sex 96 @ adultbrazil @ boogie_1969
@sexx_freak @leono77
@ vdsxx1 @ kaifel30 @ 1688aw @europstars
@ swo 22 …



# 4) Count a Word Frequency List

In [9]:
from collections import Counter

token_counter = Counter()
for text in truncated_data: # Tweets
    tokenized = pipeline.process(text)
    tokens = tokenized.split() # after segmenter, we can do whitespace splitting
    token_counter.update(tokens)

print("Most common tokens:",token_counter.most_common(20))
print("Vocabulary size:",len(token_counter))

Most common tokens: [('@', 7482), (':', 6620), ('rt', 5795), ('.', 3919), ('the', 3654), (',', 3028), ('to', 2884), ('#', 2705), ('i', 2612), ('a', 2577), ('you', 2238), ('and', 2160), ('…', 1740), ('of', 1678), ('is', 1606), ('for', 1603), ('in', 1597), ('it', 1287), ('!', 1199), ('-', 1146)]
Vocabulary size: 26513


## Stop Words

In [10]:
import nltk
nltk.download('stopwords') # download the stopwords dataset
from nltk.corpus import stopwords

# take 150 most common words from the Tweets corpus and filter out stop words and punctuation
filtered_tokens = []
punctuation_chars = '. , : ( ) ! ? " = & - ; ... \\ '.split() # list of punctuation symbols to ignore
for word,count in token_counter.most_common(150):
    if word.lower() in stopwords.words("english") or word in punctuation_chars:
        continue
    filtered_tokens.append((word,count))
print("Number of tokens:",len(filtered_tokens))
print("Tokens:",filtered_tokens)

Number of tokens: 67
Tokens: [('@', 7482), ('rt', 5795), ('#', 2705), ('…', 1740), ("'s", 803), ('’s', 557), ('christmas', 528), ("n't", 483), ('like', 452), ('n’t', 408), ('one', 387), ('amp', 382), ('love', 374), ('new', 348), ('people', 337), ('get', 309), ('year', 307), ('day', 300), ('“', 280), ("'", 277), ('2017', 254), ('today', 254), ('time', 252), ('good', 250), ('1', 247), ('”', 233), ('got', 218), ('see', 215), ('know', 202), ('back', 198), ('want', 198), ('best', 194), ('need', 192), ("'m", 191), ('2', 188), ('happy', 187), ('family', 187), ('2018', 185), ('u', 183), ('make', 179), ('’m', 173), ('go', 170), ('life', 169), ('thank', 168), ('!!', 164), ('much', 163), ('bts', 162), ('would', 161), ('/', 160), ('merry', 159), ('first', 157), ('follow', 156), ('us', 154), ('great', 152), ('trump', 151), ('really', 151), ('ca', 145), ('right', 144), ("'re", 144), ('even', 141), ('everyone', 141), ('think', 140), ('could', 139), ('let', 139), ('..', 136), ('someone', 134), ('ever'

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 5) Calculate *tf-idf*

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = truncated_data

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

print(vectorizer.get_feature_names())
print(vectorizer.get_stop_words())
print(X.shape)
print(X)

['00', '000', '000th', '0014', '006', '00am', '00pm', '01', '0123456789', '01am', '01l', '02', '02_sochie', '02m', '03', '031', '0327coolkids', '05', '06', '0625', '06m', '07', '08', '0807', '09', '0f1e6d16', '0inches', '0jbarker', '0ptmshwang', '0x06a336b88d72f471bb54438c42ecb1f9f3cf7df7', '0xcl0ud9', '10', '100', '1000', '10000x', '1004', '100beautifulwomenof2017', '100k', '100m', '100mostbe', '100mostbeautifulfaces2017', '100racistthings', '101', '102', '1039', '104', '105', '105s9', '1080', '10882', '109', '10am', '10k', '10x', '11', '111', '1110', '111th', '115k', '11a', '11am', '11pm', '11th', '11월', '12', '120', '1200', '120k', '120million', '120x217', '122', '124', '1240', '1242', '12460', '125', '125m', '129', '12am', '12daysofchristmas', '12p', '12pm', '12shiawasseefish', '12v', '13', '135', '136064', '139', '13elievesg', '13mins', '13s', '13th', '14', '140', '140p', '1491380l', '14c', '14fishstreet', '14froger14', '14s', '14th', '15', '150', '1500', '1500espnjudd', '151', '1

# 6) Similar Tweets

In [12]:
import os
import re
import binascii
from time import time

In [13]:
def get_shingles(truncated_data,k=10):
    """Get all shingles from requested file (hashes of these shingles)
    """
    for line in truncated_data:
        tweets = ' '.join([line[:-1].strip() for line in truncated_data])
        L = len(tweets)
        shingles = set()  # we use a set to automatically eliminate duplicates
        for i in range(L-k+1):
            shingle = tweets[i:i+k]
            crc = binascii.crc32(shingle.encode("utf-8")) #& 0xffffffff  # hash the shingle to a 32-bit integer
            shingles.add(crc)
        return list(shingles)

In [14]:
shingles = get_shingles(truncated_data,k=5)
shingles

[222298115,
 720896004,
 1497366533,
 3554148356,
 908066827,
 2528116752,
 1968701460,
 1387790357,
 555745301,
 623378455,
 704118805,
 3462397983,
 3880255531,
 4187488301,
 1680343096,
 199229500,
 890765374,
 4175429695,
 3759669312,
 1633681473,
 2854748227,
 1946157123,
 3251109960,
 4290248777,
 1528823883,
 646971469,
 4002938960,
 389546064,
 1949827153,
 1033896020,
 1649410136,
 2722627676,
 3890741342,
 2914517086,
 3940548705,
 2563244132,
 1197998181,
 1032323177,
 3038249069,
 3778543728,
 2579497074,
 2696413303,
 4264034423,
 1075314810,
 245366906,
 2651324542,
 306708612,
 1409810564,
 3628073099,
 318767245,
 3872915598,
 3454009489,
 1308098707,
 1045430421,
 3733979293,
 3268935841,
 2260205730,
 3638558883,
 2198864036,
 2071462051,
 957874340,
 443023528,
 1573034,
 4183818414,
 3292528815,
 2454716597,
 520618171,
 969932988,
 1551368388,
 710410438,
 316145862,
 2457338054,
 935854283,
 541065423,
 1286078672,
 832569555,
 4195352788,
 4289200341,
 63963351,


## using minshahing to find similar tweets

In [15]:
# set global parameters to process the whole dataset
bands = 5
rows = 5
nsig = bands*rows  # number of elements in signature, or the number of different random hash functions
maxShingleID = 2**32-1  # record the maximum shingle ID that we assigned
nextPrime = 4294967311  # next prime number after maxShingleID
A = np.random.randint(0,nextPrime,size=(nsig,))
B = np.random.randint(0,nextPrime,size=(nsig,))

def minhash_vectorized(shingles,A,B,nextPrime,maxShingleID,nsig):
    signature = np.ones((nsig,))*(maxShingleID+1)
    for ShingleID in shingles:
        hashCodes = ((A*ShingleID+B) % nextPrime) % maxShingleID
        np.minimum(signature, hashCodes, out=signature)
    return signature

In [ ]:

t = time()
signatures_all_files_1 = []
for sh in get_shingles(truncated_data, k=5):
    signature = minhash_vectorized(get_shingles(truncated_data, k=5),A,B, nextPrime, maxShingleID, nsig)
    signatures_all_files_1.append(signature)
t1 = time()-t
print("slow code took {} seconds".format(t1))
t = time()
signatures_all_files_2 = []
for sh in get_shingles(truncated_data, k=5):
    signature = minhash_vectorized(sh, A, B, nextPrime, maxShingleID, nsig)
    signatures_all_files_2.append(signature)
t2 = time()-t
print("slow code took {} seconds".format(t2))
print('speedup {}'.format(t1/t2))
signatures_all_files_1 = np.array(signatures_all_files_1)
signatures_all_files_2 = np.array(signatures_all_files_2)
print("results are the same: {}".format(np.allclose(signatures_all_files_1, signatures_all_files_2)))